### Converting Models via ONNX 
This script is designed to be the end-to-end master for converting models from one framework to another. The following frameworks are included:
* Keras
* Matlab (future)
* PyTorch
* Scikit-Learn
* TensorFlow

This scipt is to act as a function with the following parameters:
**Input(s):**
1. Model file name (provide absolute path if not within current dir)
2. 'frameworkIn' format
3. 'frameworkOut' format

**Output(s):**
1. Model in ONNX format
2. Model in 'frameworkOut' format

**Future Work:**
* Create folders for *modelONNX* and *modelOut* files
* Validate front and backend work with onnx==1.6.0 and onnx_tf==1.6.0
* Display completion time of entire convertion

In [1]:
# User Inputs:
frameworkIn = "onnx"    # {keras | matlab | onnx | pytorch | scikit-learn | tensorflow}
frameworkOut = "keras"

#pathIn = "cifar_entire.pth"
onnxPath = 'output_ONNX/'
outPath = 'output/'

netronView = True

In [2]:
# Base Imports
import netron
import pathlib
import os

# ONNX
import onnx
import onnxruntime

# Tensorflow
#import tensorflow as tf

print("---- 'ONNX_Master.ipynb' ---- \n")

---- 'ONNX_Master.ipynb' ---- 



In [3]:
# Local py scripts:
import convertToONNX
import convertFromONNX
import util

Package Version:
numpy:		 1.19.2 
tensorflow:	 2.2.0 



In [4]:
print("Package Versions", "\n"
      #"numpy:\t\t", numpy.__version__, '\n'
      "onnx:\t\t", onnx.__version__, '\n'
      "onnxruntime:\t", onnxruntime.__version__, '\n'
      #"skl2onnx:\t", skl2onnx.__version__, "\n"
      #"tensorflow:\t", tf.__version__, '\n'
      #"torch:\t\t", torch.__version__, "\n"
      #"torchvision:\t", torchvision.__version__, "\n"
     )

Package Versions 
onnx:		 1.6.0 
onnxruntime:	 1.5.2 



In [5]:
# Debug helper - assumes all models are local 
if frameworkIn == "keras":
    pathIn = "mnist-model.h5"
elif frameworkIn == "onnx":
    #pathIn = "mnist_tf.onnx"
    pathIn = "vgg16.onnx"
elif frameworkIn == "pytorch":
    pathIn = "vgg16.pth"
elif frameworkIn == "scikit-learn":
    pathIn = "iris-model.pkl"
elif frameworkIn == "tensorflow":
    pathIn = "MNIST_tf.pb"              # Not working
    pathIn = "mnist_tf.h5"
else:
    print("ERROR: Invalid framework!!")

### Input File Handling

In [6]:
# Get import file
pathIn = pathlib.Path(pathIn)
modelIn = pathIn.name
modelONNX, baseName = util.ChangeExtension(modelIn, 'onnx') 

# Check for output directories provided by user
util.CheckDirectories(onnxPath, outPath)
modelONNX = onnxPath + modelONNX

print("File path: ", pathIn, "\n\n"
      "Model imported:\t\t", modelIn, "\n"
      "Model converted:\t", modelONNX)

# Make sure modelIn is located within current working directory
util.CopyToDirectory(pathIn, modelIn)

File path:  vgg16.onnx 

Model imported:		 vgg16.onnx 
Model converted:	 output_ONNX/vgg16.onnx


### Convert to ONNX

In [7]:
# Run conversion script
print("Converting model: ", frameworkIn, " to ONNX...\n")
    
# Import framework specific packages  
if frameworkIn == "keras":
    convertToONNX.Keras_ONNX(modelIn, modelONNX)           # Tested
elif frameworkIn == "pytorch":
    convertToONNX.PyTorch_ONNX(modelIn, modelONNX)         # Tested 
elif frameworkIn == "onnx":
    print("Model already in ONNX format!")
elif frameworkIn == "scikit-learn":
    convertToONNX.ScikitLearn_ONNX(modelIn, modelONNX)     # Tested
elif frameworkIn == "tensorflow":
    convertToONNX.TensorFlow_ONNX(modelIn, modelONNX)      # Tested (h5)
else:
    print("Invalid framework chosen.")

print("Conversion complete!\n", "-"*65)

Converting model:  onnx  to ONNX...

Model already in ONNX format!
Conversion complete!
 -----------------------------------------------------------------


### Load and Verify ONNX Model

In [8]:
# Load ONNX Model
onnxIn = onnx.load(modelONNX)

# Check that IR is well formed
onnx.checker.check_model(onnxIn)
print(modelONNX, " has been loaded and checked!")

# Run inference using ONNX Runtime
convertToONNX.ONNX_Inference(modelONNX)

# Use Netron to view and verify model
if netronView:
    print("Loading Netron in separate tab...")
    netron.start(modelONNX, port=8081)

FileNotFoundError: [Errno 2] No such file or directory: 'output_ONNX/vgg16.onnx'

### Convert to FrameworkOut

**NOTE: As of 12/8/2020 there are issues with onnx_tf==1.7.0 causing the Jupyter Notebook kernel to crash. The following is needed:**
* onnx     1.6.0
* onnx_tf  1.6.0

In [ ]:
# Converting Class (ONNX -> frameworkOut)
if frameworkOut == "keras": 
    modelOut, _ = util.ChangeExtension(modelONNX, "h5")       # 
    convertFromONNX.ONNX_Keras(onnxIn, modelOut)
elif frameworkOut == "onnx":                     
    print("ONNX file already created!")
elif frameworkOut == "pytorch": 
    modelOut, _ = util.ChangeExtension(modelONNX, "pt")       #
    convertFromONNX.ONNX_PyTorch(onnxIn, modelOut)
elif frameworkOut == "scikit-learn": 
    modelOut, _ = util.ChangeExtension(modelONNX, "pb")       #
    convertFromONNX.ONNX_ScikitLearn(onnxIn, modelOut), 
elif frameworkOut == "tensorflow": 
    modelOut, _ = util.ChangeExtension(modelONNX, "tf")       # Tested (doesn't work on ALL models)
    
    print("WARNING: There are currently bugs in onnx-tf 1.6.0 and 1.7.0")
    convertFromONNX.ONNX_TensorFlow(onnxIn, modelOut),
    
print("Conversion complete!")
print("-"*65)